### Домашнє завдання: Пониження розмірностей для Аналізу Портретів Клієнтів

#### Контекст
В цьому ДЗ ми попрацюємо з методами пониження розмірності на наборі даних для задачі аналізу портретів клієнтів (Customer Personality Analysis). **В попередньому ДЗ ми працювали з цими даними використовуючи кластеризацію, зараз використаємо кластеризацію і візуалізауємо результати з різними методами.**

Customer Personality Analysis - це аналіз різних сегментів клієнтів компанії. Цей аналіз дозволяє бізнесу краще розуміти своїх клієнтів і полегшує процес адаптації продуктів під конкретні потреби, поведінку та інтереси різних типів клієнтів.

Аналіз портретів клієнтів допомагає бізнесу змінювати свій продукт на основі цільової аудиторії, розділеної на різні сегменти. Наприклад, замість того, щоб витрачати гроші на маркетинг нового продукту для всіх клієнтів у базі даних компанії, бізнес може проаналізувати, який сегмент клієнтів найімовірніше придбає продукт, і потім зосередити маркетингові зусилля лише на цьому сегменті.

#### Вхідні дані
Вам надано набір даних з такими атрибутами:

**Характеристики користувачів:**
- `ID`: Унікальний ідентифікатор клієнта
- `Year_Birth`: Рік народження клієнта
- `Education`: Рівень освіти клієнта
- `Marital_Status`: Сімейний стан клієнта
- `Income`: Річний дохід домогосподарства клієнта
- `Kidhome`: Кількість дітей у домогосподарстві клієнта
- `Teenhome`: Кількість підлітків у домогосподарстві клієнта
- `Dt_Customer`: Дата реєстрації клієнта у компанії
- `Recency`: Кількість днів з моменту останньої покупки клієнта
- `Complain`: 1, якщо клієнт скаржився за останні 2 роки, 0 - якщо ні

**Продукти:**
- `MntWines`: Сума, витрачена на вино за останні 2 роки
- `MntFruits`: Сума, витрачена на фрукти за останні 2 роки
- `MntMeatProducts`: Сума, витрачена на м'ясні продукти за останні 2 роки
- `MntFishProducts`: Сума, витрачена на рибні продукти за останні 2 роки
- `MntSweetProducts`: Сума, витрачена на солодощі за останні 2 роки
- `MntGoldProds`: Сума, витрачена на золото за останні 2 роки

**Акції:**
- `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
- `AcceptedCmp1`: 1, якщо клієнт прийняв пропозицію у першій кампанії, 0 - якщо ні
- `AcceptedCmp2`: 1, якщо клієнт прийняв пропозицію у другій кампанії, 0 - якщо ні
- `AcceptedCmp3`: 1, якщо клієнт прийняв пропозицію у третій кампанії, 0 - якщо ні
- `AcceptedCmp4`: 1, якщо клієнт прийняв пропозицію у четвертій кампанії, 0 - якщо ні
- `AcceptedCmp5`: 1, якщо клієнт прийняв пропозицію у п'ятій кампанії, 0 - якщо ні
- `Response`: 1, якщо клієнт прийняв пропозицію в останній кампанії, 0 - якщо ні

**Взаємодія з компанією:**
- `NumWebPurchases`: Кількість покупок, зроблених через вебсайт компанії
- `NumCatalogPurchases`: Кількість покупок, зроблених за каталогом
- `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
- `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць


Для початку, запустіть код нижче. Всі ці кроки ми робили в попередньому ДЗ і для того, щоб результати кластеризації у нас були схожими, потрібно аби передобробка була однаковою.

In [22]:
import pandas as pd

# 1. Завантаження даних
df = pd.read_csv('marketing_campaign.csv', sep='\t')

# 2. Обробка пропущених значень
df['Income_not_filled'] = df.Income.isna()
df.Income = df.Income.fillna(-1)

# 3. Обробка дати реєстрації
df.Dt_Customer = pd.to_datetime(df.Dt_Customer, format='%d-%m-%Y')
today = df.Dt_Customer.max()
df['days_lifetime'] = (today - df.Dt_Customer).dt.days
df['years_customer'] = df.Year_Birth.apply(lambda x: today.year - x)

# 4. Категоризація рівня освіти
df_education = pd.get_dummies(df.Education, prefix='education').astype(int)
df = pd.concat([df, df_education], axis=1)

# 5. Очищення сімейного стану
marital_status_map = {'Alone': 'Single', 'Absurd': 'Else', 'YOLO': 'Else'}
df['Marital_Status_clean'] = df.Marital_Status.map(marital_status_map)
df_ms = pd.get_dummies(df.Marital_Status_clean, prefix='marital').astype(int)
df = pd.concat([df, df_ms], axis=1)

# 6. Форматування доходу і видалення викиду
df.Income = df.Income.astype(int)
df = df[df.Income != 666666]

# 7. Створення фінального набору даних
X = df.drop(['ID', 'Dt_Customer', 'Education', 'Marital_Status', 'Marital_Status_clean'], axis=1)
X.reset_index(drop=True, inplace=True)

### Завдання 1: Виконання кластеризації та пониження розмірності для візуалізації результатів

Ваше завдання — провести кластеризацію клієнтів та візуалізувати результати кластеризації, використовуючи метод головних компонент (PCA) для пониження розмірності даних.

#### Інструкції:

1. **Вибір ключових характеристик:**
   Давайте обмежимось тільки наступними хараткеристиками для кластеризації цього разу:
   - `Income`: Річний дохід домогосподарства клієнта
   - `Recency`: Кількість днів з моменту останньої покупки клієнта
   - `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
   - `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
   - `days_lifetime`: Кількість днів з моменту реєстрації клієнта у компанії
   - `years_customer`: Вік клієнта
   - `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць
   Відберіть в наборі даних `X` лише ці характеристики.

2. **Нормалізація даних:**
   Використайте метод `MinMaxScaler` для нормалізації значень обраних характеристик.

3. **Кластеризація:**
   Проведіть кластеризацію клієнтів, використовуючи метод `KMeans` з трьома кластерами.

4. **Пониження розмірності:**
   Використайте метод головних компонент (PCA) для пониження розмірності даних до трьох компонент.

5. **Візуалізація результатів:**
   Використовуючи plolty express побудуйте 3D-графік розподілу клієнтів у просторі трьох головних компонент, де кольором позначено кластери.

6. **Опишіть, що спостерігаєте:**
   Чи кластеризація чітко розділила дані?

Далі ми детальніше проінтерпретуємо результати візуалізації і пониження розмірностей.

In [23]:
# Вибір ключових характеристик для кластеризації
features = [
    'Income', 'Recency', 'NumStorePurchases',
    'NumDealsPurchases', 'days_lifetime', 'years_customer',
    'NumWebVisitsMonth'
]
X_cluster = X[features]
X_cluster.head()

,Income,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth
0,58138,58,4,3,663,57,7
1,46344,38,2,2,113,60,5
2,71613,26,10,1,312,49,4
3,26646,26,4,2,139,30,6
4,58293,94,6,5,161,33,5


In [24]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Нормалізація даних
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_cluster)

# Кластеризація за допомогою KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

In [25]:
X_cluster = X[features].copy()
X_cluster['cluster'] = clusters

In [26]:
X_cluster

,Income,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth,cluster
0,58138,58,4,3,663,57,7,2
1,46344,38,2,2,113,60,5,0
2,71613,26,10,1,312,49,4,1
3,26646,26,4,2,139,30,6,0
4,58293,94,6,5,161,33,5,2
...,...,...,...,...,...,...,...,...
2234,61223,46,4,2,381,47,5,0
2235,64014,56,5,7,19,68,7,2
2236,56981,91,13,1,155,33,6,1
2237,69245,8,10,2,156,58,3,1


In [27]:
# Зниження розмірності до трьох компонент
pca = PCA(n_components=3, random_state=0)
X_pca = pca.fit_transform(X_scaled)

# Перетворення у DataFrame для візуалізації
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
pca_df['cluster'] = clusters
pca_df.head()


,PC1,PC2,PC3,cluster
0,0.380238,-0.093351,-0.247944,2
1,-0.422398,0.107615,-0.130675,0
2,-0.071943,-0.238744,0.349257,1
3,-0.414681,-0.039905,-0.103791,0
4,0.012613,0.501960,0.112838,2


In [28]:
import plotly.express as px

# Побудова 3D-графіку розподілу кластерів
fig = px.scatter_3d(
    pca_df, x='PC1', y='PC2', z='PC3',
    color=pca_df['cluster'].astype(str),
    title='3D PCA Кластеризація клієнтів',
    labels={'cluster': 'Кластер'}
)
fig.show()

На графіку добре помітна відносна відокремленість кластерів у просторі головних компонент.

Це говорить про те, що PCA вдало зменшило розмірність і при цьому зберегло основні закономірності, які дозволяють відрізнити клієнтів за певними характеристиками.

Аналіз кластерів:


Кластер 0 (червоний):
Чітко відокремлений від інших кластерів. Має мінімальне перекриття з зеленими і синіми точками.

Це може свідчити, що клієнти з цього кластеру мають значно відмінні характеристики.

Кластер 1 (зелений) і Кластер 2 (синій):
Також добре розділені, але знаходяться ближче один до одного, ніж до кластеру 0.Їхнє часткове перекриття може свідчити про меншу різницю між цими групами.


### Завдання 2: Аналіз результатів пониження розмірності

1. **Розрахунок частки поясненої дисперсії:**
   Визначте, яка частка загальної варіації даних пояснюється кожною з трьох головних компонент (PC1, PC2, PC3) за допомогою атрибуту `explained_variance_ratio_` об'єкта PCA. Виведіть результат на екран.

2. **Розрахунок кумулятивної частки поясненої дисперсії:**
   Обчисліть кумулятивну частку поясненої дисперсії для трьох головних компонент, щоб зрозуміти, скільки варіації даних пояснюється першими кількома компонентами.

In [29]:
import numpy as np
np.sum(pca.explained_variance_ratio_)

np.float64(0.8399120325811188)

In [30]:
# Частка поясненої дисперсії кожною з трьох головних компонент
explained_variance = pca.explained_variance_ratio_
print("Частка поясненої дисперсії для кожної компоненти:")
for i, var in enumerate(explained_variance, 1):
    print(f"PC{i}: {var:.4f}")

Частка поясненої дисперсії для кожної компоненти:
PC1: 0.3020
PC2: 0.2867
PC3: 0.2512


Ми отримали наступні частки поясненої дисперсії для кожної компоненти:
- PC1: 0.3020
- PC2: 0.2867
- PC3: 0.2512

Перша компонента (PC1) є найінформативнішою, але різниця з другою і третьою компонентами порівняно невелика, що свідчить про відносно рівномірний внесок.

In [31]:
# Кумулятивна частка поясненої дисперсії
cumulative_variance = explained_variance.cumsum()
print("\nКумулятивна частка поясненої дисперсії:")

for i, var in enumerate(cumulative_variance, 1):
    print(f"Після додавання PC{i}: {var:.4f}")


Кумулятивна частка поясненої дисперсії:
Після додавання PC1: 0.3020
Після додавання PC2: 0.5887
Після додавання PC3: 0.8399


Кумулятивна дисперсія складає:


PC1: 30.20%, після додавання PC2: 58.87%, після додавання PC3: 83.99%

Тобто, перші три головні компоненти разом пояснюють приблизно 83.99% усієї варіації в даних. Це непоганий результат, оскільки значну частину інформації вдалося зберегти навіть після зниження розмірності.

### Завдання 3: Інтерпретація "Loadings"

Продовжуємо інтерпретацію результатів `PCA`і познайомимось з новим поняттям `loadings`, яке допоможе нам знайти звʼязок між головними компонентами і оригінальними ознаками в наборі даних.

Ми зараз побудували візуалізацію кластерів точок даних в просторі трьох головних компонент. Але хочеться знайти звʼязок між головними компонентами і оригінальними ознаками. Для розуміння, які початкові характеристики даних мають найбільший вплив на ці головні компоненти, ми можемо використати атрибут `components_` методу `PCA`.

#### Що таке `pca.components_`?

`pca.components_` — це масив, який містить коефіцієнти (або "ваги"), що показують внесок кожної вихідної ознаки у кожну з головних компонент. Ці коефіцієнти ще називаються **"loading"** або "навантаженнями" компонент.

- **Loadings** (`навантаження`) відображають важливість кожної змінної (ознаки) для відповідної головної компоненти. Вони показують, яким чином змінні поєднуються, щоб утворити нові, зменшені вимірювання.
- Якщо коефіцієнт має високе абсолютне значення (як позитивне, так і негативне), це вказує на те, що відповідна змінна сильно впливає на головну компоненту.

#### Саме завдання
Ваше завдання — обчислити "навантаження" для кожної з головних компонент і інтерпретувати результати.

1. **Обчислення loadings для компонент:**
   Використайте атрибут `components_` об'єкта PCA для створення DataFrame, який відображатиме внесок кожної вихідної ознаки в кожну головну компоненту.

2. **Інтерпретація результатів:**
   Виведіть значення "навантажень" і проаналізуйте, які ознаки найбільше впливають на кожну головну компоненту.

In [32]:
# Створюємо DataFrame з "навантаженнями" (loadings)
loadings = pd.DataFrame(pca.components_, columns= features, index=['PC1', 'PC2', 'PC3'])

# Виводимо результати
loadings

,Income,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth
PC1,0.063557,0.475786,0.284282,0.103049,0.821912,0.012658,0.049530
PC2,-0.047082,0.878876,-0.187195,-0.059324,-0.431061,0.001022,-0.031377
PC3,0.373826,0.029910,0.816668,-0.050687,-0.305490,0.080411,-0.300089


PC1 (Перша головна компонента). Найбільший внесок у PC1 мають ознаки days_lifetime (0.82), тобто кількість днів з моменту реєстрації та Recency (0,48) - кількість днів з останньої покупки, ткож важливою є NumStorePurchases (0.28) — кількість покупок у фізичних магазинах.

PC1 відображає часові аспекти та регулярність взаємодії клієнтів з компанією.​


PC2 (Друга головна компонента) має високий позитивний внесок від Recency = 0.88 та негативний від days_lifetime = -0.43.

Ця компонента показує різницю між клієнтами, які з компанією давно, і тими, хто приєднався нещодавно або недавно щось купував.


PC3 (Третя головна компонента) показує, наскільки клієнти надають перевагу покупкам в офлайн магазинах.

Основна змінна — NumStorePurchases (0.82) — кількість покупок в магазинах.

Також виділяються Income (0.37), NumWebVisitsMonth (-0.30), і days_lifetime (-0.31).


###Завдання 4
Давайте проаналізуємо "навантаження" (**loadings**) для трьох головних компонент після вилучення ознаки `Income`. Це допоможе нам зрозуміти, як змінилася важливість інших ознак для кожної головної компоненти, коли одна з ключових характеристик (`Income`) була вилучена.

#### Кроки для проведення аналізу і ваше завдання:

1. Видаліть ознаку `Income` з нашого набору даних `X` і повторно виконайте PCA (метод головних компонент) для отримання нових "навантажень".

2. Обчисліть нові "навантаження" для трьох головних компонент на наборі даних без `Income`

3. Проаналізуйте, які ознаки мають найбільший вплив на кожну головну компоненту після вилучення `Income`.

4. Перегляньте, наскільки кожна з головних компонент пояснює дисперсію в даних без ознаки `Income`.

In [33]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px

# Визначаємо список ознак, виключаючи 'Income'
selected_features = [
    'Recency', 'NumStorePurchases', 'NumDealsPurchases',
    'days_lifetime', 'years_customer', 'NumWebVisitsMonth'
]

# Створюємо новий DataFrame з обраними ознаками
X_selected = X[selected_features].copy()

# Масштабуємо дані
scaler = MinMaxScaler()
X_scaled_selected = scaler.fit_transform(X_selected)

# Кластеризація за допомогою KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
clusters_1 = kmeans.fit_predict(X_scaled_selected)

# Додаємо мітки кластерів до DataFrame
X_selected['cluster'] = clusters_1


In [34]:
# Застосовуємо PCA для зниження розмірності до трьох компонент
pca = PCA(n_components=3, random_state=0)
X_pca = pca.fit_transform(X_scaled_selected)

# Створюємо DataFrame з результатами PCA
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
pca_df['cluster'] = clusters

# Візуалізація результатів кластеризації у 3D
fig = px.scatter_3d(
    pca_df, x='PC1', y='PC2', z='PC3',
    color=pca_df['cluster'].astype(str),
    title='3D PCA Кластеризація клієнтів (без Income)',
    labels={'cluster': 'Кластер'}
)
fig.show()

In [35]:
import numpy as np
np.sum(pca.explained_variance_ratio_)

np.float64(0.8614145570202412)

In [36]:
# Аналіз "навантажень" (loadings) головних компонент
loadings = pd.DataFrame(
    pca.components_,
    columns=selected_features,
    index=['PC1', 'PC2', 'PC3']
)
loadings

,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth
PC1,0.500712,0.226953,0.105012,0.825793,0.007110,0.069051
PC2,0.865392,-0.145537,-0.065833,-0.472036,0.005913,-0.052233
PC3,0.008648,0.924549,-0.027582,-0.232275,0.079808,-0.289919


In [37]:
# Частка поясненої дисперсії для кожної компоненти
explained_variance = pca.explained_variance_ratio_
print("\nЧастка поясненої дисперсії для кожної компоненти:")
for i, var in enumerate(explained_variance, 1):
    print(f"PC{i}: {var:.4f}")


Частка поясненої дисперсії для кожної компоненти:
PC1: 0.3215
PC2: 0.3055
PC3: 0.2345


In [38]:
# Кумулятивна частка поясненої дисперсії
cumulative_variance = explained_variance.cumsum()
print("\nКумулятивна частка поясненої дисперсії:")
for i, var in enumerate(cumulative_variance, 1):
    print(f"До PC{i}: {var:.4f}")


Кумулятивна частка поясненої дисперсії:
До PC1: 0.3215
До PC2: 0.6269
До PC3: 0.8614


Інтерпретація по компонентах:


PC1 (32.15%) — досвід клієнта. Найбільше пов’язана з days_lifetime (0.83) — чим довше клієнт із компанією, тим вищий показник. Recency (0.50) — нещодавня активність також впливає, але менше. Інші змінні мають мінімальний вплив. Ця компонента відображає тривалість взаємодії клієнта з компанією.


PC2 (30.55%) — найбільше навантаження має Recency (0.87), days_lifetime — від'ємне (-0.47). Ця компонента протиставляє "нових" та "старих" клієнтів.

PC3 (23.45%) — характеризує тип покупок: офлайн vs онлайн.
NumStorePurchases (0.92) — ключова змінна. Вплив також мають: NumWebVisitsMonth (-0.29) — більше онлайн-відвідувань, менше офлайн-покупок; days_lifetime (-0.23) — "новіші" клієнти менш активні в офлайн-магазинах.

Ця компонента показує, хто більше купує офлайн і менше взаємодіє онлайн.

Загальний висновок:
PCA без Income дозволяє чітко виділити такі аспекти:

PC1: Скільки часу клієнт із компанією.

PC2: Як давно клієнт був активний.

PC3: Чи клієнт купує більше в магазині чи онлайн.


PC1 та PC2 стали більш значущими — пояснюють більшу частку варіації. Це означає, що без доходу більше навантаження припадає на поведінкові ознаки.

Загальне охоплення трьома компонентами вище (86.14%), отже, модель краще відображає структуру клієнтів без урахування доходу.

### Завдання 5: Візуалізація кластеризації за допомогою t-SNE

Ваше завдання — використати метод t-SNE для візуалізації результатів кластеризації клієнтів у двовимірному просторі. Метод t-SNE допомагає знизити розмірність даних та зберегти локальні структури в даних, що робить його ефективним для візуалізації високорозмірних даних. Ми також зможемо порівняти результат цього методу з РСА.

1. Використайте метод t-SNE для зниження розмірності до 2х вимірів даних, які включають ознаки всі, що і в завданні 1, а також були відмасштабовані перед пониженням розмірностей.

2. Створіть новий DataFrame з координатами, отриманими після застосування t-SNE, та додайте до нього мітки кластерів.

3. Побудуйте інтерактивний 2D-графік розподілу клієнтів, де кольором буде позначено різні кластери і проаналізуйте графік з рекомендаціями нижче (можливо треба буде вивести додаткові візуалізації чи таблиці для інтерпретації, але треба прям зрозуміти, які ознаки формують який кластер і чим кластери відрізняються одне від одного).

  **Опишіть отримані кластери з точки зору ознак.**

4. Опишіть відмінність графіка tSNE від PCA.

#### ЯК можна інтерпретувати з t-SNE?

Хоча t-SNE не надає "компонентів" як РСА, він забезпечує низьковимірне представлення даних, яке можна візуально інтерпретувати:

- **Кластери:** t-SNE особливо добре показує кластери подібних точок. Якщо ви бачите чітко визначені кластери на графіку t-SNE, це свідчить про наявність груп схожих спостережень у ваших даних. Проаналізувати їх можемо, якщо додамо дані в `hover_data` або якщо якісь з даних виведемо як розмір чи форма точок на візуалізації. Також корисно може бути вивести середні значення ознак по кластерам.
- **Локальна структура:** Відносне розташування точок одного кластеру на графіку t-SNE може допомогти вам зрозуміти, які дані подібні між собою.
- **Глобальна структура:** Будьте обережні; t-SNE менш надійний для відображення глобальних структур (наприклад, відстаней між кластерами) у порівнянні з PCA, бо t-SNE націлений на збереження саме локальних структур.

In [39]:
from sklearn.manifold import TSNE

# Застосування t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X_scaled)

tsne_df = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2'])
tsne_df['cluster'] = clusters

# 6. Візуалізація t-SNE
fig_tsne = px.scatter(
    tsne_df, x='TSNE1', y='TSNE2',
    color=tsne_df['cluster'].astype(str),
    title='t-SNE Візуалізація Кластерів Клієнтів',
    labels={'cluster': 'Кластер'},
    hover_data={'cluster': True}
)
fig_tsne.show()

In [40]:
# Середні значення ознак по кластерам
X_cluster_summary = X_cluster.groupby('cluster').mean()
print("Середні значення ознак для кожного кластеру:")
print(X_cluster_summary)

Середні значення ознак для кожного кластеру:
               Income    Recency  NumStorePurchases  NumDealsPurchases  \
cluster                                                                  
0        44990.324561  21.352130           4.264411           2.327068   
1        69681.154622  51.163025          10.275630           2.273950   
2        44622.521277  73.878251           4.078014           2.356974   

         days_lifetime  years_customer  NumWebVisitsMonth  
cluster                                                    
0           337.989975       43.987469           5.849624  
1           398.705882       46.610084           3.880672  
2           336.508274       45.346336           5.822695  


Кластер 0

- Income: середній (~45K)

- Recency: низький (≈21) – часті покупки, активні останнім часом

- NumStorePurchases: невелика кількість покупок в офлайні (≈4.26)

- NumWebVisitsMont: найвища активність онлайн (≈5.85)

Цей кластер можна охарактеризувати як сегмент клієнтів із середнім рівнем доходу, які активно взаємодіють з компанією онлайн. Вони становлять перспективну цільову аудиторію для цифрових маркетингових кампаній.

Кластер 1

- Income: найвищий (~70K)

- Recency: середній (≈51) – помірна активність

- NumStorePurchases: найбільше покупок офлайн(≈10.28)

- NumWebVisitsMont: найменша кількість відвідувань сайту (≈3.88)

- days_lifetime: найдовше з компанією

Висновок: це постійні клієнти, взаємодіють переважно офлайн, мають високий дохід. Найцінніші для VIP-кампаній, програми лояльності.

Кластер 2

- Income: середній (~44.6K)

- Recency: високий (~74) – давно не взаємодіяли

- NumStorePurchases: низький (≈4.08)

- NumWebVisitsMont: трохи нижчий, але схоже з кластером 0 (≈5.82)

Висновок: пасивний сегмент аудиторії, який потребує додаткової уваги для повернення до активності.


t-SNE досить чітко розділив 3 кластери, видно досить чіткі межі між групами.

Порівняння PCA і t-SNE.

У нашому випадку:

PC1 (30.2%) + PC2 (28.7%) + PC3 (25.1%) разом пояснюють ~84% варіації, тобто PCA доволі добре справляється.

Проте на графіку PCA все ж спостерігається часткове перекриття між кластерами — тобто деякі клієнти в просторі компонент розміщені близько до інших кластерів, і не завжди можна провести чітку межу.

Це свідчить про те, що структура даних може бути нелінійною, і звичайне лінійне зменшення розмірності (як у PCA) не повністю захоплює внутрішні зв'язки.

Ціль t-SNE - максимальне збереження локальних зв’язків між об'єктами.

В нашій візуалізації з t-SNE кластери досить добре розділені, тобто клієнти мають помітні відмінності у своїй поведінці чи характеристиках.

t-SNE краще показує розділення кластерів клієнтів та дає змогу візуально виокремити групи для таргетингу або аналізу.

PCA дає розуміння загальної структури варіацій у даних, але менш ефективний у візуальному розділенні кластерів.
